In [1]:
import jsonlines
from datasets import load_dataset

/root/autodl-tmp/conda_envs/autoif/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
outputs = []

In [3]:
ultraint = load_dataset("openbmb/UltraInteract_sft", split='train').shuffle(seed=42).select(range(115000))

In [4]:
for idx, data in enumerate(ultraint):
    outputs.append({
        "id": f"{idx}_UltraInteract_sft_{data['id']}", 
        "msg": [{"role": "user", "content": data["instruction"]}],
        "original_ans": [{"role": "assistant", "content": data["response"]}]
    })

In [5]:
onemilorca = load_dataset("Open-Orca/1million-gpt-4", split='train').shuffle(seed=42).select(range(115000))

In [6]:
for idx, data in enumerate(onemilorca):
    outputs.append({
        "id": f"{idx}_1million-gpt-4_{data['id']}", 
        "msg": [{"role": "system", "content": data["system_prompt"]},{"role": "user", "content": data["question"]}],
        "original_ans": [{"role": "assistant", "content": data["response"]}]
    })

In [7]:
slimorca = load_dataset("Open-Orca/SlimOrca", split='train').shuffle(seed=42).select(range(115000))

In [ ]:
err_idx = []
for idx, data in enumerate(slimorca):
    try:
        outputs.append({
            "id": f"SlimOrca_{idx}", 
            "msg": [{"role": "system", "content": data["conversations"][0]["value"]},{"role": "user", "content": data["conversations"][1]["value"]}],
            "original_ans": [{"role": "assistant", "content": data["conversations"][2]["value"]}]
        })
    except IndexError:
        outputs.append({
            "id": f"SlimOrca_{idx}", 
            "msg": [{"role": "system", "content": data["conversations"][0]["value"]},{"role": "user", "content": data["conversations"][1]["value"]}],
            "original_ans": ["idxerr"]
        })
        print(f"IndexError for idx {idx} in SlimOrca dataset: {data['conversations']}")
        err_idx.append(idx)

In [9]:
oasst2 = load_dataset("sablo/oasst2_curated", split='train')

In [10]:
oasst2[0]

{'messages': [{'content': 'I am making mayonnaise, it was starting to thicken but now it has become runny and liquid again, is there any way to salvage it?',
   'role': 'user'},
  {'content': "Yes, it's possible to fix runny mayonnaise! The most common reason for mayonnaise becoming runny is because the oil was added too quickly or the egg yolk wasn't emulsified properly. Here are some steps you can take to fix it:\n\n1. Separate another egg yolk and place it in a clean, dry bowl.\n2. Slowly add the runny mayonnaise to the egg yolk while whisking vigorously.\n3. Once all the runny mayonnaise has been added, continue whisking until the mixture has emulsified and thickened.\n4. If the mayonnaise is still too runny, you can add another egg yolk and repeat the process.\n\nIf the mayonnaise still won't thicken, you can try adding a small amount of dijon mustard or vinegar to the mixture, which can act as emulsifiers and help stabilize the mayonnaise. It's important to add these ingredients 

In [11]:
for idx, data in enumerate(oasst2):
    outputs.append({
        "id": f"oasst2_{idx}", 
        "msg": [{"role": "user", "content": data["messages"][0]["content"]}],
        "original_ans": [{"role": "assistant", "content": data["messages"][1]["content"]}]
    })

In [12]:
with jsonlines.open("/root/autodl-tmp/projs/AutoIF/make_inst_following_deepseek/instf_volc_deepseek_with_original_ans_350k.jsonl", "w") as f:
    for output in outputs:
        output_final = {
            "custom_id": output['id'], 
            "body": {
                "messages": output['msg'],
                "max_tokens": 16000,
            },
            "original_ans": output['original_ans'],
        }
        f.write(output_final)

